In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False ## 마이너스 부호 깨짐 방지

pd.set_option('display.max_columns', None)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', None)                      # 출력할 열의 너비
# pd.reset_option('display.max_rows', None)                    # 출력할 행의 최대개수  None
pd.set_option('display.unicode.east_asian_width', True) 

## sessions
사용자에 대한 웹 세션 로그
- user_id: to be joined with the column 'id' in users table
- action -> 조회, 검색결과, 대시보드 등 다양한 액션이 있음 -> click에 대한 대분류이지 않을까?
- action_type
- action_detail
- device_type -> 웹 방문에 대한 기기 종류
- secs_elapsed -> 세션에 머문 시간 (초단위)

세션 데이터세트에서 데이터는 2014년 1월 1일까지만 거슬러 올라가는 반면, 사용자 데이터세트는 2010년까지 거슬러 올라갑니다.

In [ ]:
from action_detail_category import detail
new_ss = detail(sessions)
new_ss

In [81]:
## sessions
sessions = pd.read_csv('D:\데브코스_최종프로젝트/airbnb-recruiting-new-user-bookings/sessions.csv')
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [82]:
sessions.isnull().sum()

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

## user_id
- 결측치 - 34496 존재 -> train 데이터와 id로 결합하기 어려움으로 삭제하는 것이 좋아보임

In [83]:
## user_id 결측치 삭제
sessions.dropna(subset=['user_id'], inplace=True)

In [84]:
sessions.isnull().sum()

user_id                0
action             79480
action_type      1122957
action_detail    1122957
device_type            0
secs_elapsed      135483
dtype: int64

## action_detail 대분류 진행 -> Action? 

In [85]:
## 원본 데이터 보존
new_sessions = sessions.copy()

## Nan값 0으로 변환
new_sessions = new_sessions.fillna(0)

In [8]:
def list_diff(a, b):
    set_a = set(a)
    set_b = set(b)
    a_ = list(set_a - set_b)
    b_ = list(set_b - set_a)
    total_ = a_ + b_
    # print(f'a에만 있고 b에는 없는 것{set_a - set_b}')
    # print(f'b에만 있고 a에는 없는 것{set_b - set_a}')
    print(total_)
    return True

# 비교 함수
def find_diff(list1, list2):
    for i in list1:
        if i not in list2:
            print(i)

In [27]:
orginlist = new_sessions['action_detail'].unique().tolist()

In [59]:
new_list = view_search + unknown + user_update + change_trip + login_out + review + host + booking + new_user + message + wishlist

### 검색/조회

In [86]:
# unavailable_dates -> 여행 특성 변경으로 고려
view_search = ['view_locations', 'view_search_results', 'p3', 'unavailable_dates', 'p5', 'p4', 'p1', 'popular_wishlists', 
               'terms_and_privacy', 'click_about_host', 'move_map', 'click_amenities', 'toggle_archived_thread', 
               'user_friend_recommendations', 'p4_refund_policy_terms', 'translations', 'p4_terms', 'toggle_starred_thread',
              'cancellation_policy', 'share', 'view_security_checks', 'place_worth', 'view_resolutions', 
               'view_user_real_names', 'view_ghostings', 'view_ghosting_reasons', 'homepage',
              'special_offer_field', 'tos_2014', 'your_trips', 'notifications', 'photos','profile_references', 'cancellation_policies'] 

In [87]:
new_sessions['action_detail'].replace(view_search, 'view_search', inplace=True)

### Unknown

In [88]:
unknown = ['-unknown-',]

In [89]:
new_sessions['action_detail'].replace(unknown, 'unknown', inplace=True)

### 위시리스트

In [90]:
wishlist = ['wishlist', 'wishlist_content_update', 'user_wishlists', 'wishlist_note', 'friends_wishlists', 'email_wishlist_button',
           'email_wishlist', 'airbnb_picks_wishlists']

In [91]:
new_sessions['action_detail'].replace(wishlist, 'wishlist', inplace=True)

### 정보 수정

In [92]:
user_update = ['user_profile', 'header_userpic', 'edit_profile', 'user_profile_content_update', 'phone_verification_success',
              'account_transaction_history', 'account_privacy_settings', 'user_languages', 'delete_phone_numbers',
              'change_or_alter', 'account_payment_methods', 'modify_users', 'previous_trips', 'at_checkpoint',
               'alteration_field', 'change_password', 'create_payment_instrument', 'view_identity_verifications',
              'phone_numbers', 'set_default_payment_instrument', 'deactivate_user_account',
              'delete_payment_instrument', 'update_user', 'create_phone_numbers', 'update_user_profile', 
             'account_notification_settings', 'request_new_confirm_email', 'account_payout_preferences', 'profile_verifications']

In [93]:
new_sessions['action_detail'].replace(user_update, 'user_update', inplace=True)

### 여행 특성 변경

In [94]:
change_trip = ['change_trip_characteristics', 'trip_availability']

In [95]:
new_sessions['action_detail'].replace(change_trip, 'change_trip', inplace=True)

### 로그인/로그아웃
- 'oauth_response', -> 네이버 등  간편 로그인 기능도 OAuth2 프로토콜 기반의 사용자 인증 기능을 제공



In [96]:
login_out = ['user_social_connections', 'login', 'oauth_response', 'oauth_login', 'login_page', 'login_modal', 'forgot_password', 'set_password', 
             'set_password_page', 'confirm_email']

In [97]:
new_sessions['action_detail'].replace(login_out, 'login_out', inplace=True)

### 리뷰 조회

In [98]:
review = ['user_reviews', 'profile_reviews', 'click_reviews']

In [99]:
new_sessions['action_detail'].replace(review, 'review', inplace=True)

### 호스트

In [100]:
host = ['similar_listings', 'listing_reviews', 'update_listing', 'dashboard', 'user_listings', 'your_listings', 'listing_recommendations', 'update_listing_description',
       'manage_listing', 'view_listing', 'list_your_space', 'create_listing', 'listing_descriptions', 'listing_reviews_page',
        'translate_listing_reviews', 'delete_listing_description', 'delete_listing', 'calculate_worth', 'change_availability',
       'host_respond', 'admin_templates', 'host_guarantee', 'user_tax_forms', 'host_home', 'respond_to_alteration_request', 
        'host_standard_suspension', 'host_respond_page', 'remove_dashboard_alert', 'post_checkout_action']

In [101]:
new_sessions['action_detail'].replace(host, 'host', inplace=True)

### 예약
- 예약 보류, 예약 취소

In [102]:
booking = ['instant_book', 'coupon_code_click', 'change_contact_host_dates', 'reservations', 'payment_instruments',
           'pending', 'apply_coupon', 'book_it', 'apply_coupon_click', 'request_to_book',
          'your_reservations','guest_receipt', 'guest_itinerary', 'complete_booking', 'booking', 'guest_cancellation', 
           'apply_coupon_click_success', 'modify_reservations', 'view_reservations', 'alteration_request', 'create_alteration_request', 
           'host_refund_guest', 'cancellation_policy_click','apply_coupon_error', 'coupon_field_focus']

# 'guest_cancellation'(취소), 'modify_reservations'(애매), 'cancellation_policy_click'(취소) 'host_refund_guest' -> 환불이기 때문에 추후 분석해보기?

In [103]:
new_sessions['action_detail'].replace(booking, 'booking', inplace=True)

### 회원가입

In [104]:
new_user = ['confirm_email_link', 'create_user', 'signup', 'signup_login_page', 'signup_modal', 'read_policy_click']

In [105]:
new_sessions['action_detail'].replace(new_user, 'new_user', inplace=True)

### 메시지

In [106]:
message = ['contact_host', 'message_thread', 'message_post', 'message_inbox', 'send_message','message_to_host_focus', 'lookup_message_thread', 'message_to_host_change']

In [107]:
new_sessions['action_detail'].replace(message, 'message', inplace=True)

## EDA

In [110]:
new_sessions['action_detail'].value_counts()

action_detail
view_search    3459178
host           1263115
user_update    1124168
0              1122957
unknown        1028222
wishlist        872076
change_trip     494909
login_out       416955
message         364817
booking         186869
new_user        148335
review           51640
Name: count, dtype: int64

In [113]:
new_sessions[new_sessions['action_detail'] == 0]['action'].unique()

array(['lookup', 'show', 'track_page_view', 'widget', 'uptodate', 'index',
       'campaigns', 'currencies', 'localization_settings', 'update',
       'phone_verification', 'satisfy', 'track_activity', 'check',
       'signed_out_modal', 'disaster_action', 'similar_listings_v2',
       'braintree_client_token'], dtype=object)